In [ ]:
import torch
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from torch import nn, optim
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# CIFAR-10 normalization
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load the dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

# Separate cats and dogs (classes 3 and 5)
cats_dogs_train = Subset(train_dataset, [i for i, (_, label) in enumerate(train_dataset) if label in [3, 5]])
cats_dogs_test = Subset(test_dataset, [i for i, (_, label) in enumerate(test_dataset) if label in [3, 5]])

# Separate other classes for autoencoder training
other_classes_train = Subset(train_dataset, [i for i, (_, label) in enumerate(train_dataset) if label not in [3, 5]])

# Data loaders
batch_size = 64
train_loader_cats_dogs = DataLoader(cats_dogs_train, batch_size=batch_size, shuffle=True)
test_loader_cats_dogs = DataLoader(cats_dogs_test, batch_size=batch_size, shuffle=False)
train_loader_other = DataLoader(other_classes_train, batch_size=batch_size, shuffle=True)

In [ ]:
class BasicCNN(nn.Module):
    def __init__(self):
        super(BasicCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 8 * 8, 64),
            nn.ReLU(),
            nn.Linear(64, 2)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

# Initialize and train the CNN
cnn = BasicCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn.parameters(), lr=0.001)

# Training loop
def train_model(model, loader, epochs):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in loader:
            labels = (labels == 3).long()  # Convert to binary classification (3 = cat, 5 = dog)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(loader)}")

# Train the CNN
train_model(cnn, train_loader_cats_dogs, epochs=10)


In [ ]:
class DenoisingAutoencoder(nn.Module):
    def __init__(self):
        super(DenoisingAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, kernel_size=2, stride=2),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# Initialize the autoencoder
autoencoder = DenoisingAutoencoder()
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

# Train the autoencoder
def train_autoencoder(model, loader, epochs):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, _ in loader:
            noisy_inputs = inputs + torch.randn_like(inputs) * 0.1
            noisy_inputs = torch.clip(noisy_inputs, 0., 1.)
            optimizer.zero_grad()
            outputs = model(noisy_inputs)
            loss = criterion(outputs, inputs)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(loader)}")

# Train the autoencoder
train_autoencoder(autoencoder, train_loader_other, epochs=10)

In [ ]:
class FineTuneModel(nn.Module):
    def __init__(self, encoder):
        super(FineTuneModel, self).__init__()
        self.encoder = encoder
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 8 * 8, 64),
            nn.ReLU(),
            nn.Linear(64, 2)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.fc_layers(x)
        return x

# Initialize fine-tune model with pre-trained encoder
fine_tune_model = FineTuneModel(autoencoder.encoder)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(fine_tune_model.parameters(), lr=0.001)

# Fine-tune the model
train_model(fine_tune_model, train_loader_cats_dogs, epochs=10)